In [6]:
import joblib
from collections import Counter
from helper.helper import format_number_and_round
from preprocessing.scarpping_component import extract_component_by_images
import os
import pandas as pd
import dlib
import cv2
from feature_extraction.poc import POC
from feature_extraction.vektor import Vektor
from feature_extraction.quadran import Quadran
from preprocessing.input_to_image import get_frames_by_input_video

# Load the models and label encoders
model_hidung = joblib.load('models/knn_model_dengan_hidung.joblib')
label_encoder_hidung = joblib.load('models/label_encoder_dengan_hidung.joblib')

model_tanpa_hidung = joblib.load('models/knn_model_tanpa_hidung.joblib')
label_encoder_tanpa_hidung = joblib.load('models/label_encoder.joblib')

# Video directory
# video_directory = 'dataset/demo/01_EP02_01f_H2.avi'
# video_directory = 'dataset/demo/19_EP19_03_SU1.avi'
# video_directory = 'dataset/demo/25_EP09_02_D1.avi'
video_directory = 'dataset/demo/26_EP15_01_SD_1.avi'
output_image_directory = 'output_image_5x5'

# dapatkan frame dari video
fps = cv2.VideoCapture(video_directory).get(cv2.CAP_PROP_FPS)

print(f"file video: {video_directory} dengan fps: {fps}")

# Convert video to images
gambar = get_frames_by_input_video(video_directory, output_image_directory, framePerSecond=fps)

components_setup = {
    'mulut': {
        'object_name': 'mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    'mata_kiri': {
        'object_name': 'eye_left',
        'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'mata_kanan': {
        'object_name': 'eye_right',
        'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'alis_kiri': {
        'object_name': 'eyebrow_left',
        'object_rectangle': {"x_right": 21, "x_left": 17, "y_highest": 18, "y_lowest": 21},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'alis_kanan': {
        'object_name': 'eyebrow_right',
        'object_rectangle': {"x_right": 26, "x_left": 22, "y_highest": 25, "y_lowest": 22},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'hidung_kanan': {
        'object_name': 'nose_right',
        'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    },
    'hidung_kiri': {
        'object_name': 'nose_left',
        'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    }
}

# Initialize variables for feature extraction
blockSize = 5
frames_data_all_component = []
frames_data_quadran_column = ['sumX', 'sumY', 'Tetha', 'Magnitude', 'JumlahQuadran']
quadran_dimensions = ['Q1', 'Q2', 'Q3', 'Q4']
frames_data = {component_name: [] for component_name in components_setup}
total_blocks_components = {component_name: 0 for component_name in components_setup}
data_blocks_first_image = {component_name: None for component_name in components_setup}
index = {component_name: 0 for component_name in components_setup}

# Load face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

# Function to calculate the prediction results
def get_calculate_from_predict(list_decoded_predictions):
    prediction_counts = Counter(list_decoded_predictions)
    total_predictions = len(list_decoded_predictions)
    result_prediction = None
    most_common_count = 0
    list_predictions = []

    for category, count in prediction_counts.items():
        percentage = (count / total_predictions) * 100
        list_predictions.append({
            "name": category,
            "count": count,
            "percentage": format_number_and_round(percentage)
        })
        if count > most_common_count:
            most_common_count = count
            result_prediction = category

    return result_prediction, list_predictions

# Process each image
for filename in os.listdir(output_image_directory):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        image = cv2.imread(os.path.join(output_image_directory, filename))
        image = cv2.resize(image, (600, 500))
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        rects = detector(gray)

        if len(rects) == 0:
            print(f"Skip {filename} karena tidak ada wajah yang terdeteksi")
            continue

        for component_name in components_setup:
            if index[component_name] != 0:
                frame_data_all_component = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                frame_data_quadran = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                        
        for rect in rects:
            shape = predictor(gray, rect)
            for component_name, component_info in components_setup.items():
                sum_data_by_quadran = {}
                frame_data = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                for column in frames_data_quadran_column:
                    sum_data_by_quadran[column] = {quadrant: 0 for quadrant in quadran_dimensions}

                data_blocks_image_current = extract_component_by_images(
                    image=image,
                    shape=shape,
                    frameName=filename.split(".")[0],
                    objectName=component_info['object_name'],
                    objectRectangle=component_info['object_rectangle'],
                    pixelShifting=component_info['pixel_shifting'],
                    objectDimension=component_info['object_dimension']
                )

                if data_blocks_first_image[component_name] is None:
                    frames_data[component_name].append(frame_data)
                    frame_data['Folder Path'] = "data_test"
                    frame_data['Label'] = "data_test"
                    data_blocks_first_image[component_name] = data_blocks_image_current
                    continue

                initPOC = POC(data_blocks_first_image[component_name], data_blocks_image_current, blockSize)
                valPOC = initPOC.getPOC() 

                initQuiv = Vektor(valPOC, blockSize)
                quivData = initQuiv.getVektor()

                initQuadran = Quadran(quivData) 
                quadran = initQuadran.getQuadran()

                for i, quad in enumerate(quadran):
                    frame_data[f'X{i+1}'] = quad[1]
                    frame_data[f'Y{i+1}'] = quad[2]
                    frame_data[f'Tetha{i+1}'] = quad[3]
                    frame_data[f'Magnitude{i+1}'] = quad[4]

                    frame_data_all_component[f'{component_name}-X{i+1}'] = quad[1]
                    frame_data_all_component[f'{component_name}-Y{i+1}'] = quad[2]
                    frame_data_all_component[f'{component_name}-Tetha{i+1}'] = quad[3]
                    frame_data_all_component[f'{component_name}-Magnitude{i+1}'] = quad[4]

                    if quad[5] in quadran_dimensions:
                        sum_data_by_quadran['sumX'][quad[5]] += quad[1]
                        sum_data_by_quadran['sumY'][quad[5]] += quad[2]
                        sum_data_by_quadran['Tetha'][quad[5]] += quad[3]
                        sum_data_by_quadran['Magnitude'][quad[5]] += quad[4]
                        sum_data_by_quadran['JumlahQuadran'][quad[5]] += 1

                frames_data[component_name].append(frame_data)
                frame_data['Folder Path'] = "data_test"
                frame_data['Label'] = "data_test"

                for quadran in quadran_dimensions:
                    for feature in frames_data_quadran_column:
                        column_name = f"{component_name}{feature}{quadran}"
                        frame_data_quadran[column_name] = sum_data_by_quadran[feature][quadran]

        if index[component_name] != 0:
            frames_data_all_component.append(frame_data_all_component)
            frame_data_all_component['Folder Path'] = "data_test"
            frame_data_all_component['Label'] = "data_test"

        index[component_name] += 1

# Function to predict and print results
def predict_and_print_results(df, model, label_encoder, feature_columns):
    df = df.drop(columns=feature_columns)
    predictions = model.predict(df.values)
    decoded_predictions = label_encoder.inverse_transform(predictions)
    result_prediction, list_predictions = get_calculate_from_predict(decoded_predictions)
    
    for pred in list_predictions:
        print(f"Kategori: {pred['name']}, Jumlah: {pred['count']}, Persentase: {pred['percentage']:.2f}%")
    
    print(f"\nHasil Prediksi: {result_prediction}")

df_fitur_all = pd.DataFrame(frames_data_all_component)
except_feature_columns = ['Frame', 'Folder Path', 'Label']

# Predict and print results with nose
print("Predictions with Nose:")
predict_and_print_results(df_fitur_all.copy(), model_hidung, label_encoder_hidung, except_feature_columns)

# Predict and print results without nose (remove nose features)
nose_features = [col for col in df_fitur_all.columns if 'hidung' in col]
df_fitur_tanpa_hidung = df_fitur_all.drop(columns=nose_features)

print("\nPredictions without Nose:")
predict_and_print_results(df_fitur_tanpa_hidung.copy(), model_tanpa_hidung, label_encoder_tanpa_hidung, except_feature_columns)


file video: dataset/demo/26_EP15_01_SD_1.avi dengan fps: 200.0
Predictions with Nose:
Kategori: Sadness, Jumlah: 122, Persentase: 97.60%
Kategori: Happiness, Jumlah: 2, Persentase: 1.60%
Kategori: Surprise, Jumlah: 1, Persentase: 0.80%

Hasil Prediksi: Sadness

Predictions without Nose:
Kategori: Sadness, Jumlah: 117, Persentase: 93.60%
Kategori: Happiness, Jumlah: 7, Persentase: 5.60%
Kategori: Surprise, Jumlah: 1, Persentase: 0.80%

Hasil Prediksi: Sadness


In [2]:
import joblib
from collections import Counter
import os
import pandas as pd
import dlib
import cv2
from helper.helper import format_number_and_round
from preprocessing.scarpping_component import extract_component_by_images
from feature_extraction.poc import POC
from feature_extraction.vektor import Vektor
from feature_extraction.quadran import Quadran
from preprocessing.input_to_image import get_frames_by_input_video

# Load the models and label encoders
model_hidung = joblib.load('models/knn_model_dengan_hidung.joblib')
label_encoder_hidung = joblib.load('models/label_encoder_dengan_hidung.joblib')

model_tanpa_hidung = joblib.load('models/knn_model_tanpa_hidung.joblib')
label_encoder_tanpa_hidung = joblib.load('models/label_encoder.joblib')

# Emotions directories
emotions = {
    'Happiness': '01_EP02_01f_H2.avi',
    'Surprise': '19_EP19_03_SU1.avi',
    'Disgust': '25_EP09_02_D1.avi',
    'Sadness': '26_EP15_01_SD_1.avi'
}
base_directory = 'dataset/demo/'

output_image_directory = 'output_image_5x5'
components_setup = {
    'mulut': {
        'object_name': 'mouth',
        'object_rectangle': {"x_right": 54, "x_left": 48, "y_highest": 52, "y_lowest": 57},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 5},
        'object_dimension': {'width': 140, 'height': 40}
    },
    'mata_kiri': {
        'object_name': 'eye_left',
        'object_rectangle': {"x_right": 39, "x_left": 36, "y_highest": 38, "y_lowest": 41},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'mata_kanan': {
        'object_name': 'eye_right',
        'object_rectangle': {"x_right": 45, "x_left": 42, "y_highest": 43, "y_lowest": 47},
        'pixel_shifting': {"pixel_x": 25, "pixel_y": 25},
        'object_dimension': {'width': 90, 'height': 55}
    },
    'alis_kiri': {
        'object_name': 'eyebrow_left',
        'object_rectangle': {"x_right": 21, "x_left": 17, "y_highest": 18, "y_lowest": 21},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'alis_kanan': {
        'object_name': 'eyebrow_right',
        'object_rectangle': {"x_right": 26, "x_left": 22, "y_highest": 25, "y_lowest": 22},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": 15},
        'object_dimension': {'width': 110, 'height': 40}
    },
    'hidung_kanan': {
        'object_name': 'nose_right',
        'object_rectangle': {"x_right": 31, "x_left": 40, "y_highest": 40, "y_lowest": 48},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    },
    'hidung_kiri': {
        'object_name': 'nose_left',
        'object_rectangle': {"x_right": 47, "x_left": 35, "y_highest": 47, "y_lowest": 54},
        'pixel_shifting': {"pixel_x": 15, "pixel_y": -25},
        'object_dimension': {'width': 70, 'height': 40}
    }
}

# Initialize variables for feature extraction
blockSize = 5
frames_data_all_component = []
frames_data_quadran_column = ['sumX', 'sumY', 'Tetha', 'Magnitude', 'JumlahQuadran']
quadran_dimensions = ['Q1', 'Q2', 'Q3', 'Q4']
frames_data = {component_name: [] for component_name in components_setup}
total_blocks_components = {component_name: 0 for component_name in components_setup}
data_blocks_first_image = {component_name: None for component_name in components_setup}
index = {component_name: 0 for component_name in components_setup}

# Load face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

result_list = []

# Function to calculate the prediction results
def get_calculate_from_predict(list_decoded_predictions):
    prediction_counts = Counter(list_decoded_predictions)
    total_predictions = len(list_decoded_predictions)
    result_prediction = None
    most_common_count = 0
    list_predictions = []

    for category, count in prediction_counts.items():
        percentage = (count / total_predictions) * 100
        list_predictions.append({
            "name": category,
            "count": count,
            "percentage": format_number_and_round(percentage)
        })
        if count > most_common_count:
            most_common_count = count
            result_prediction = category

    return result_prediction, list_predictions

# Process each video file
for emotion, file in emotions.items():
    video_path = os.path.join(base_directory, file)
    
    # Convert video to images
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    print(f"Processing {video_path} with {fps} FPS")
    images = get_frames_by_input_video(video_path, output_image_directory, framePerSecond=fps)

    for filename in os.listdir(output_image_directory):
        if filename.endswith(".jpg") or filename.endswith(".png"): 
            image = cv2.imread(os.path.join(output_image_directory, filename))
            image = cv2.resize(image, (600, 500))
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            rects = detector(gray)

            if len(rects) == 0:
                print(f"Skip {filename} karena tidak ada wajah yang terdeteksi")
                continue

            for component_name in components_setup:
                if index[component_name] != 0:
                    frame_data_all_component = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                    frame_data_quadran = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}

            for rect in rects:
                shape = predictor(gray, rect)
                for component_name, component_info in components_setup.items():
                    sum_data_by_quadran = {}
                    frame_data = {'Frame': f"{index[component_name] + 1}({filename.split('.')[0]})"}
                    for column in frames_data_quadran_column:
                        sum_data_by_quadran[column] = {quadrant: 0 for quadrant in quadran_dimensions}

                    data_blocks_image_current = extract_component_by_images(
                        image=image,
                        shape=shape,
                        frameName=filename.split(".")[0],
                        objectName=component_info['object_name'],
                        objectRectangle=component_info['object_rectangle'],
                        pixelShifting=component_info['pixel_shifting'],
                        objectDimension=component_info['object_dimension']
                    )

                    if data_blocks_first_image[component_name] is None:
                        frames_data[component_name].append(frame_data)
                        frame_data['Folder Path'] = os.path.join(base_directory, emotion, 'Video_Image_RAW 200fps', filename)
                        frame_data['Label'] = emotion
                        data_blocks_first_image[component_name] = data_blocks_image_current
                        continue

                    initPOC = POC(data_blocks_first_image[component_name], data_blocks_image_current, blockSize)
                    valPOC = initPOC.getPOC() 

                    initQuiv = Vektor(valPOC, blockSize)
                    quivData = initQuiv.getVektor()

                    initQuadran = Quadran(quivData) 
                    quadran = initQuadran.getQuadran()

                    for i, quad in enumerate(quadran):
                        frame_data[f'X{i+1}'] = quad[1]
                        frame_data[f'Y{i+1}'] = quad[2]
                        frame_data[f'Theta{i+1}'] = quad[3]
                        frame_data[f'Magnitude{i+1}'] = quad[4]

                    if index[component_name] % 50 == 0:
                        print(f"Processing frame {index[component_name] + 1} of {filename}")

                    frames_data[component_name].append(frame_data)
                    frames_data_all_component.append(frame_data_all_component)

            index[component_name] += 1

    # Save predictions
    print(f"Processing {emotion}")
    data_frame_all_component = pd.DataFrame(frames_data_all_component)

    predicted_labels_hidung = []
    predicted_labels_tanpa_hidung = []

    for data in frames_data_all_component:
        features = [data[f'{component_name}-X{i+1}'] for component_name in components_setup for i in range(4)]
        prediction_hidung = model_hidung.predict([features])
        prediction_tanpa_hidung = model_tanpa_hidung.predict([features])

        decoded_prediction_hidung = label_encoder_hidung.inverse_transform(prediction_hidung)
        decoded_prediction_tanpa_hidung = label_encoder_tanpa_hidung.inverse_transform(prediction_tanpa_hidung)

        predicted_labels_hidung.append(decoded_prediction_hidung[0])
        predicted_labels_tanpa_hidung.append(decoded_prediction_tanpa_hidung[0])

    result_hidung, result_labels_hidung = get_calculate_from_predict(predicted_labels_hidung)
    result_tanpa_hidung, result_labels_tanpa_hidung = get_calculate_from_predict(predicted_labels_tanpa_hidung)

    result_list.append({
        'Emotion': emotion,
        'Result_Hidung': result_hidung,
        'Labels_Hidung': result_labels_hidung,
        'Result_Tanpa_Hidung': result_tanpa_hidung,
        'Labels_Tanpa_Hidung': result_labels_tanpa_hidung
    })

# Output results
print(result_list)


Processing dataset/demo/01_EP02_01f_H2.avi with 200.0 FPS
width_object: 140, height_object: 40
width_object: 90, height_object: 55
width_object: 90, height_object: 55
width_object: 110, height_object: 40
width_object: 110, height_object: 40
width_object: 70, height_object: 40
width_object: 70, height_object: 40
width_object: 140, height_object: 40
Processing frame 1 of frame10.jpg
width_object: 90, height_object: 55
Processing frame 1 of frame10.jpg
width_object: 90, height_object: 55
Processing frame 1 of frame10.jpg
width_object: 110, height_object: 40
Processing frame 1 of frame10.jpg
width_object: 110, height_object: 40
Processing frame 1 of frame10.jpg
width_object: 70, height_object: 40
Processing frame 1 of frame10.jpg
width_object: 70, height_object: 40
width_object: 140, height_object: 40
Processing frame 1 of frame100.jpg
width_object: 90, height_object: 55
Processing frame 1 of frame100.jpg
width_object: 90, height_object: 55
Processing frame 1 of frame100.jpg
width_object: 

KeyError: 'mulut-X1'